In [1]:
import numpy as np
from scipy.stats import norm
import importlib
import helper_functions
importlib.reload(helper_functions)
from helper_functions import get_option_info

In [2]:
def binomial_option_price(S0, K, T, r, sigma, N, option_type='call', american=False):
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)

    # Stock prices at maturity
    ST = np.array([S0 * (u ** j) * (d ** (N - j)) for j in range(N + 1)])

    # Payoffs at maturity
    if option_type == 'call':
        V = np.maximum(ST - K, 0)
    else:
        V = np.maximum(K - ST, 0)

    # Backward induction
    for i in range(N - 1, -1, -1):
        V = np.exp(-r * dt) * (p * V[1:] + (1 - p) * V[:-1])
        if american:
            ST = S0 * (u ** np.arange(i + 1)) * (d ** (i - np.arange(i + 1)))
            if option_type == 'call':
                V = np.maximum(V, ST - K)
            else:
                V = np.maximum(V, K - ST)

    return V[0]

In [3]:
api="<your api key here>"
S0,K,r,sigma,T,mid_price,bid,ask=get_option_info("AAPL",api,True)

In [4]:
# Example usage of binomial
bin_price = binomial_option_price(S0, K, T, r, sigma, N=1000)

In [5]:
if bin_price>=bid and bin_price<=ask:
    print("Black-Scholes-Merton model is within range of market bid and ask")
else:
    print("Black-Scholes-Merton model is not within the range of market bid and ask")
print("bid: ",bid)
print("ask: ",ask)
print("mid (model price should be close to this): ",mid_price)
print("model price: ",bin_price)

Black-Scholes-Merton model is within range of market bid and ask
bid:  162.65
ask:  165.4
mid (model price should be close to this):  164.025
model price:  163.4820854884024
